In [74]:
import os
filepath = r'D:/Data Science/GIT Projects/End-to-End-Image-Classification'
os.chdir(filepath)
%pwd

'D:\\Data Science\\GIT Projects\\End-to-End-Image-Classification'

In [75]:
from dataclasses import dataclass
from pathlib import Path

In [76]:
@dataclass(frozen=True)
class Callback_Config:
    root_dir: Path
    model_checkpoint: Path
    tensorboard_dir: Path

In [77]:
from cnn_image_classifier.constants import *
from cnn_image_classifier.utils.common import read_yaml_file,create_directories

In [ ]:
class Config_Manager:
    def __init__(self, config_file_path: Path = config_path, param_file_path: Path = param_path):
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(param_file_path)
        create_directories([self.config.artifact_root])

    def get_callbacks_config(self)-> Callback_Config:
        config = self.config.callbacks
        model_checkpoint_dir = os.path.dirname(config.model_checkpoint_filepath)
        create_directories([config.tensorboard_dir,model_checkpoint_dir])
        callback_config = Callback_Config(
            root_dir = Path(config.root_dir),
            model_checkpoint = Path(config.model_checkpoint_filepath),
            tensorboard_dir = Path(config.tensorboard_dir))

        return callback_config


In [79]:
import tensorflow as tf
import time


In [ ]:
class Prepare_Callback:
    def __init__(self, config : Callback_Config):
        self.config = config

    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y_%m_%d-%H:%M:%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_dir,f'tb_logs_at_{timestamp}')
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def create_checkpoint_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath = self.config.model_checkpoint,
                                                  save_best_only = True,)
    
    def get_callbacks(self):
        return [self.create_tb_callback, self.create_checkpoint_callback]


In [81]:
try:
    config = Config_Manager()
    callback_config = config.get_callbacks_config()
    callback = Prepare_Callback(config = callback_config)
    callback_list = callback.get_callbacks()
except Exception as e:
    raise e

[2025-02-25 15:56:55,194 : INFO : common] - yaml file:config.yaml has been read successfully.
[2025-02-25 15:56:55,203 : INFO : common] - yaml file:params.yaml has been read successfully.
[2025-02-25 15:56:55,208 : INFO : common] - directory:artifacts has been created successfully.
[2025-02-25 15:56:55,234 : INFO : common] - directory:artifacts/callbacks/tensorboard has been created successfully.
[2025-02-25 15:56:55,234 : INFO : common] - directory:artifacts/callbacks/checkpoints has been created successfully.
